<a href="https://colab.research.google.com/github/fasterinnerlooper/Notebooks/blob/main/training/train-codeberta-on-lcc_csharp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch
import torch
if not torch.cuda.is_available():
  %pip install tensorflow==2.14 intel-extension-for-tensorflow[xpu]==2.14 intel-optimization-for-horovod==0.28.1.1 torch==2.0.1 torchvision==0.15.2 intel_extension_for_pytorch==2.0.100 oneccl-bind-pt==2.0.0 --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/ -q


In [2]:
%pip install wandb numpy datasets transformers[torch] accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, whic

In [1]:
use_wandb = True

if use_wandb:
    import wandb

    wandb.init(project="codeBERTa-on-csharp")

wandb: Currently logged in as: shafiq-jetha. Use `wandb login --relogin` to force relogin


In [2]:
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
)


model_name = "fasterinnerlooper/codeBERTa-csharp"

# download prepare the data
# dataset = load_dataset("code_search_net","javascript")
dataset = load_dataset("fasterinnerlooper/LCC_csharp")
tokenizer = AutoTokenizer.from_pretrained(model_name, return_special_tokens_mask=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/838k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/482k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [3]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True,
)

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(30000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(10000))

In [6]:
small_train_dataset.features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [7]:
import os

model = AutoModelForMaskedLM.from_pretrained(model_name)
model.to("cuda")

if wandb:
    os.environ["WANDB_PROJECT"] = "codeBERTa-on-csharp"
    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "false"


training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=500,
    # push_to_hub=True,
)

# pass "wandb" to the 'report_to' parameter to turn on wandb logging
if wandb:
    training_args.report_to = ["wandb"]

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/334M [00:00<?, ?B/s]

In [8]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.475200,1.818951
2,1.396200,1.718245


Epoch,Training Loss,Validation Loss
1,1.475200,1.818951
2,1.396200,1.718245
3,1.413300,1.652988


TrainOutput(global_step=11250, training_loss=1.449346421983507, metrics={'train_runtime': 2764.9579, 'train_samples_per_second': 32.55, 'train_steps_per_second': 4.069, 'total_flos': 2984110571520000.0, 'train_loss': 1.449346421983507, 'epoch': 3.0})

In [9]:
model.push_to_hub("codeBERTa-csharp")
tokenizer.push_to_hub("codeBERTa-csharp")

model.safetensors:   0%|          | 0.00/334M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fasterinnerlooper/codeBERTa-csharp/commit/ffc3b5e5f102558cd7b8a81d455e0c410c5df319', commit_message='Upload tokenizer', commit_description='', oid='ffc3b5e5f102558cd7b8a81d455e0c410c5df319', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

if wandb:
    wandb.finish()

Perplexity: 5.34


eval/loss,█▄▁▂
eval/runtime,███▁
eval/samples_per_second,▁▁▁█
eval/steps_per_second,▁▁▁█
train/epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇█████
train/global_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇█████
train/learning_rate,██▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁
train/loss,█▇▆▅▅▄▄▃▃▂▂▂▁▂▂▂▂▁▁▂▂▂
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [17]:
from transformers import pipeline

text = """
//
// Author:
//   Jb Evain (jbevain@gmail.com)
//
// Copyright (c) 2008 - 2015 Jb Evain
// Copyright (c) 2008 - 2011 Novell, Inc.
//
// Licensed under the MIT/X11 license.
//

namespace Mono.Cecil.Cil {

	public sealed class VariableDefinition : VariableReference {

		public bool IsPinned {
			get { return variable_type.IsPinned; }
		}

		public VariableDefinition (TypeReference variableType)
			: base (variableType)
		{
		}

		public override VariableDefinition Resolve ()
		{
			<mask> <mask>;
		}
	}
}"""

# should produce 'return this'
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device="cuda")
res = pipe(text)
for r in res[0]:
	if(r['score'] > .50):
		for x in (pipe(r['sequence'])):
			print(str(x['score']) + ": " + str(x['token_str']))
			print(r['token_str'] + x['token_str'])

0.19661445915699005:  override
 return override
0.1217818558216095:  false
 return false
0.04463060945272446:  true
 return true
0.014115393161773682:  boot
 return boot
0.011947603896260262:  null
 return null
